<a href="https://colab.research.google.com/github/Joyschool/ktcloud_genai/blob/main/103_LLM_Tokenization_%EC%99%84%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenization



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



## [참고] Tokenization

### [실습] 예제 1: 토크나이제이션 과정 단계별 확인

In [ ]:
# 경고 메시지만 숨기기
import warnings
import os

# 1. 경고 메시지 비활성화
warnings.filterwarnings("ignore", category=UserWarning)

# 2. Hugging Face 진행률 표시 끄기
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

In [ ]:
from transformers import GPT2Tokenizer
import torch

# 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 원본 텍스트
input_text = "프로그래밍은 재미있다. 프로그래밍은"
print(f"✅ 원본 텍스트: '{input_text}'")
print()

# 1단계: 토큰으로 분할 (문자열 형태)
tokens = tokenizer.tokenize(input_text)
print(f"1️⃣ 토큰 분할 결과: {tokens}")
print()

# 2단계: 각 토큰의 ID 확인
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"2️⃣ 토큰 ID들: {token_ids}")
print()

# 3단계: encode 함수로 한번에 처리
input_ids_list = tokenizer.encode(input_text)
print(f"3️⃣ encode 결과 (리스트): {input_ids_list}")
print()

# 4단계: 텐서 형태로 변환
input_ids = tokenizer.encode(input_text, return_tensors='pt')
print(f"4️⃣ 텐서 형태로 변환")
print(f"⭢ input_ids (텐서): {input_ids}")
print(f"⭢ input_ids 형태: {input_ids.shape}")
print(f"⭢ input_ids 타입: {type(input_ids)}")
print()

# 역변환: ID를 다시 텍스트로
decoded_text = tokenizer.decode(input_ids[0])
print(f"✅ 역변환 결과: '{decoded_text}'")


### [실습] 예제 2: 다양한 텍스트의 토크나이제이션 비교

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 다양한 텍스트 예제
texts = [
    "안녕하세요",
    "Hello world",
    "프로그래밍",
    "AI는 미래다",
    "123456",
    "hello@email.com"
]

print("=== 다양한 텍스트의 토크나이제이션 결과 ===")
for text in texts:
    tokens = tokenizer.tokenize(text)
    input_ids = tokenizer.encode(text, return_tensors='pt')

    print(f"텍스트: '{text}'")
    print(f"토큰: {tokens}")
    print(f"input_ids: {input_ids.tolist()}")
    print(f"토큰 개수: {len(tokens)}")
    print("-" * 50)

### [실습] 예제 3: 토큰 ID의 사용 과정

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 모델과 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 원본 텍스트
input_text = "프로그래밍은 재미있다. 프로그래밍은"
print(f"입력 텍스트: '{input_text}'")

# 토크나이제이션
input_ids = tokenizer.encode(input_text, return_tensors='pt')
print(f"input_ids: {input_ids}")
print(f"각 ID가 나타내는 토큰:")

# 각 ID가 무슨 토큰인지 확인
for i, token_id in enumerate(input_ids[0]):
    token = tokenizer.decode([token_id])
    print(f"  위치 {i}: ID {token_id.item()} → '{token}'")

print()

# 모델에 입력하여 다음 토큰 확률 계산
with torch.no_grad():
    outputs = model(input_ids)
    # logits: [배치_크기, 시퀀스_길이, 어휘_크기]
    logits = outputs.logits

print(f"모델 출력 형태: {logits.shape}")
print(f"마지막 토큰 위치의 확률 분포 크기: {logits[0, -1, :].shape}")

# 다음 토큰으로 가능성이 높은 상위 5개 확인
last_token_logits = logits[0, -1, :]
probabilities = torch.softmax(last_token_logits, dim=-1)
top_5_prob, top_5_indices = torch.topk(probabilities, 5)

print("\n다음 토큰 예측 상위 5개:")
for i, (prob, idx) in enumerate(zip(top_5_prob, top_5_indices)):
    token = tokenizer.decode([idx])
    print(f"{i+1}. '{token}' (확률: {prob:.4f})")

### [주의!] 이상한 토큰들의 정체
출력 결과에서 GPT2는 한글을 바이트 단위로 분해 때문에 토큰의 이상하게 보여질 수 있다**⭢의미 손실 발생할 수 있다.
- 의미 단위로 분해해야 정확함
    - ✅ 원본 텍스트: '프로그래밍은 재미있다. 프로그래밍은'
    - 1️⃣ 토큰 분할 결과: ['í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë', 'ŀ', 'ĺ', 'ë', '°', 'į', 'ìĿ', 'Ģ', 'Ġì', 'ŀ', '¬', 'ë', '¯', '¸', 'ì', 'ŀ', 'Ī', 'ëĭ', '¤', '.', 'Ġ', 'í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë', 'ŀ', 'ĺ', 'ë', '°', 'į', 'ìĿ', 'Ģ']
- ['í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ' ...] 이 토큰들의 정체

|정체|설명|문제점|
|--- |--- |--- |
|UTF-8 바이트의 잘못된 해석 |한글 바이트를 Latin-1로 디코딩한 결과 |의미 완전 손실 |
|BPE 알고리즘의 한계 |영어 위주 학습으로 한글 패턴 미학습 |비효율적 토큰화 |
|어휘집 부족 |GPT-2 어휘집에 한글 토큰 거의 없음 |알 수 없는 토큰으로 처리 |

#### 예제 1: 문제 상황 정확한 분석

In [ ]:
from transformers import GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 한글 텍스트
korean_text = "프로그래밍은 재미있다"
print(f"원본 텍스트: '{korean_text}'")
print()

# 토큰 분할 결과
tokens = tokenizer.tokenize(korean_text)
print(f"토큰 개수: {len(tokens)}개")
print(f"토큰들: {tokens[:10]}... (처음 10개만 표시)")
print()

# 이상한 문자들의 정체 확인
print("=== 이상한 토큰들의 정체 ===")
for i, token in enumerate(tokens[:5]):
    # 토큰을 바이트로 변환해보기
    try:
        token_id = tokenizer.convert_tokens_to_ids([token])[0]
        print(f"토큰 {i+1}: '{token}' → ID: {token_id}")
    except:
        print(f"토큰 {i+1}: '{token}' → 변환 불가")

print()

# UTF-8 바이트 분석
print("=== UTF-8 바이트 레벨 분석 ===")
korean_bytes = korean_text.encode('utf-8')
print(f"한글 텍스트의 UTF-8 바이트: {korean_bytes}")
print(f"바이트 개수: {len(korean_bytes)}개")

# 각 바이트를 개별 문자로 디코딩 시도
print("바이트별 분석:")
for i, byte_val in enumerate(korean_bytes[:10]):
    print(f"바이트 {i+1}: {byte_val} (0x{byte_val:02x})")

#### 예제 2: 영어와 한글 토크나이제이션 비교

In [ ]:
from transformers import GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 비교 텍스트들
texts = {
    "영어": "Programming is fun",
    "한글": "프로그래밍은 재미있다",
    "숫자": "12345",
    "특수문자": "Hello! @#$%"
}

print("=== 언어별 토크나이제이션 비교 ===")
for lang, text in texts.items():
    tokens = tokenizer.tokenize(text)
    token_count = len(tokens)
    char_count = len(text)

    print(f"\n{lang}: '{text}'")
    print(f"  문자 수: {char_count}")
    print(f"  토큰 수: {token_count}")
    print(f"  효율성: {token_count/char_count:.2f} (토큰/문자)")
    print(f"  토큰 예시: {tokens[:5]}...")

#### 예제 3: 올바른 다국어 토크나이저 사용

In [ ]:
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

# 다국어 지원 토크나이저들 비교
tokenizer_models = {
    "GPT-2 (영어 전용)": "gpt2",
    "mBERT (다국어)": "bert-base-multilingual-cased",
    "XLM-RoBERTa (다국어)": "xlm-roberta-base"
}

korean_text = "프로그래밍은 재미있다"

print("=== 다양한 토크나이저 비교 ===")
for model_name, model_id in tokenizer_models.items():
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokens = tokenizer.tokenize(korean_text)

        print(f"\n{model_name}:")
        print(f"  토큰 수: {len(tokens)}")
        print(f"  토큰들: {tokens}")

        # 역변환 확인
        reconstructed = tokenizer.convert_tokens_to_string(tokens)
        print(f"  역변환: '{reconstructed}'")
        print(f"  원본과 동일: {'✅' if reconstructed.strip() == korean_text else '❌'}")

    except Exception as e:
        print(f"\n{model_name}: 로드 실패 - {e}")

### 해결 방법과 권장사항


- 한글 처리를 위한 올바른 선택

|용도 |권장 모델 |이유
|--- |--- |--- |
| 한글 텍스트 생성| GPT-3.5/4, KoGPT| 한글 데이터로 훈련됨|
| 한글 이해/분류| KoBERT, KoELECTRA| 한국어 특화 모델|
| 다국어 처리| mBERT, XLM-RoBERTa| 다국어 동시 지원|
| 실습/학습용| 영어 예제 사용| GPT-2 본래 성능 확인|

- 학습용 개선된 예제

In [ ]:
# 올바른 접근: 영어로 실습하기
from transformers import GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# GPT-2가 잘 처리하는 영어 텍스트
english_text = "Programming is fun. Programming is"
print(f"영어 텍스트: '{english_text}'")

tokens = tokenizer.tokenize(english_text)
print(f"토큰들: {tokens}")
print(f"토큰 수: {len(tokens)}")

# 각 토큰의 의미 확인
print("\n토큰별 의미:")
for i, token in enumerate(tokens):
    print(f"{i+1}. '{token}' → 의미있는 단위")



---

